In [ ]:
import google.generativeai as genai

genai.configure(api_key='GEMINI_API_KEY') # Configure with your API key
model = genai.GenerativeModel('gemini-1.5-flash') # Initialize a model, e.g., 'gemini-pro'

/home/azar/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
user_input = """
print('hello world')


for i in range(10):
    for j in range(i):
        print("*",end='')
    print('')

  
"""


In [4]:
final_prompt = f"""
You are a professional code reviewer and documentation expert. 
Your task is to take the provided code and add clear, concise, and professional comments 
that explain the logic, purpose, and functionality of the code.

Requirements:
1. Return the original code exactly as given, but with added comments in the appropriate places.
2. Do not modify variable names, function names, or the structure of the code.
3. Make comments helpful for a reader who is learning or maintaining the code.
4. Return the result strictly as a JSON object with the following format:

{{
  "code_with_comments": "...",   # The full code with comments included
  "extension": "..."             # The file extension of the code (e.g., .py for Python, .js for JavaScript, .cpp for C++)
}}

Here is the code you must work on:
{user_input}
"""


In [6]:
response = model.generate_content(final_prompt)
print(response.text)

```json
{
  "code_with_comments": "```python\n# This program first prints a greeting to the console.\nprint('hello world')\n\n\n# This nested loop structure prints a right-angled triangle pattern of asterisks to the console.\n# The outer loop iterates 10 times, controlling the number of rows in the triangle.\n# The inner loop iterates from 0 up to the current value of the outer loop variable (i), \n# printing an asterisk for each iteration. This ensures that each row has one more asterisk than the previous row.\n# \\n is added to move the cursor to next line after each row's print\nfor i in range(10):\n    for j in range(i):\n        print(\"*\",end='')\n    print('')\n\n  \n```",
  "extension": ".py"
}
```



In [9]:
import json
import re

def clean_code_json(response: str) -> dict:
    """
    Cleans a JSON-like response that may include markdown code fences and invalid characters.
    Returns a valid Python dictionary.
    """
    # Step 1: Remove BOM or weird hidden characters
    response = response.strip().lstrip("\ufeff").strip()
    
    # Step 2: Extract JSON between first { and last }
    match = re.search(r"\{.*\}", response, re.DOTALL)
    if not match:
        raise ValueError("No valid JSON object found in response")
    json_str = match.group(0)
    
    # Step 3: Parse JSON safely
    data = json.loads(json_str)
    
    # Step 4: Clean code fences from code_with_comments
    code = data.get("code_with_comments", "")
    code = re.sub(r"^```[a-zA-Z]*\n?", "", code)   # remove ```python
    code = re.sub(r"\n?```$", "", code)           # remove ending ```
    data["code_with_comments"] = code.strip()
    
    return data



cleaned = clean_code_json(response.text)
print(cleaned)


{'code_with_comments': '# This program first prints a greeting to the console.\nprint(\'hello world\')\n\n\n# This nested loop structure prints a right-angled triangle pattern of asterisks to the console.\n# The outer loop iterates 10 times, controlling the number of rows in the triangle.\n# The inner loop iterates from 0 up to the current value of the outer loop variable (i), \n# printing an asterisk for each iteration. This ensures that each row has one more asterisk than the previous row.\n# \\n is added to move the cursor to next line after each row\'s print\nfor i in range(10):\n    for j in range(i):\n        print("*",end=\'\')\n    print(\'\')', 'extension': '.py'}


In [11]:
print(cleaned['code_with_comments'])

# This program first prints a greeting to the console.
print('hello world')


# This nested loop structure prints a right-angled triangle pattern of asterisks to the console.
# The outer loop iterates 10 times, controlling the number of rows in the triangle.
# The inner loop iterates from 0 up to the current value of the outer loop variable (i), 
# printing an asterisk for each iteration. This ensures that each row has one more asterisk than the previous row.
# \n is added to move the cursor to next line after each row's print
for i in range(10):
    for j in range(i):
        print("*",end='')
    print('')


# Save the file for downloads

In [ ]:
import os
def save_code_file(data: dict, filename: str = "output"):
    """
    Saves the code_with_comments into a file in the current folder
    using the provided extension.
    
    Args:
        data (dict): Dictionary containing "code_with_comments" and "extension".
        filename (str): Base name for the output file (default: "output").
    
    Returns:
        str: Path of the created file.
    """
    if "code_with_comments" not in data or "extension" not in data:
        raise ValueError("Data must contain 'code_with_comments' and 'extension' keys.")
    
    # Construct file path
    ext = data["extension"].strip()
    if not ext.startswith("."):
        ext = "." + ext
    filepath = os.path.join(os.getcwd(), filename + ext)
    
    # Write file
    with open(filepath, "w", encoding="utf-8") as f:
        f.write(data["code_with_comments"])
    
    return filepath


file_path = save_code_file(cleaned, filename="my_script")
print(f"File saved at: {file_path}")

File saved at: /home/azar/Desktop/tiktok2.0/day4/8/my_script.py
